# Fourth Down Decision Analysis - Machine Learning Pipeline

This notebook trains and evaluates multiple ML classifiers to predict 4th down go-for-it decisions, then calculates **Coach Aggression Over Expectation (AOE)**.

## Pipeline Overview
1. Load and preprocess data from R pipeline
2. Train and evaluate multiple classifiers
3. Select best model based on F1-score
4. Hyperparameter tuning
5. Calculate Aggression Over Expectation for each coach
6. Export rankings and predictions

## 1. Setup & Imports

In [1]:
# Core libraries
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (accuracy_score, f1_score, classification_report, 
                             confusion_matrix, roc_auc_score, precision_score, recall_score)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier, 
                              AdaBoostClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

import joblib

print("Core libraries loaded successfully!")

Core libraries loaded successfully!


In [2]:
# Try to import advanced ML libraries
try:
    from xgboost import XGBClassifier
    XGBOOST_AVAILABLE = True
    print("✓ XGBoost available")
except ImportError:
    XGBOOST_AVAILABLE = False
    print("✗ XGBoost not available - install with: pip install xgboost")

try:
    from lightgbm import LGBMClassifier
    LIGHTGBM_AVAILABLE = True
    print("✓ LightGBM available")
except ImportError:
    LIGHTGBM_AVAILABLE = False
    print("✗ LightGBM not available - install with: pip install lightgbm")

✓ XGBoost available
✓ LightGBM available


## 2. Configuration

In [3]:
# Configuration parameters
RANDOM_STATE = 42
TEST_SIZE = 0.2
CV_FOLDS = 5
MIN_COACH_OPPORTUNITIES = 20  # Minimum 4th downs for coach ranking

# Features to use in modeling
FEATURE_COLUMNS = [
    # Core situation features
    'distance',
    'log_distance',
    'yards_to_goal',
    'period',
    
    # Time features
    'seconds_remaining_game',
    'seconds_remaining_half',
    
    # Score features
    'point_differential',
    'total_score',
    'is_trailing',
    'is_leading',
    'abs_point_diff',
    
    # Betting features
    'pos_team_spread',
    'pos_team_is_underdog',
    
    # Field position
    'in_opponent_territory',
    'in_red_zone',
    'in_scoring_position',
    'goal_to_go',
    
    # Down and distance categories
    'short_yardage',
    'medium_yardage',
    'long_yardage',
    
    # Talent features
    'talent_gap',
    'scaled_talent_gap',
    
    # Game situation
    'late_and_close',
    'garbage_time',
    'two_minute_drill',
    'is_conference_game',
    'pos_team_is_home',
    
    # Historical context
    'prev_play_successful',
    'prev_4th_down_success_rate',
    
    # Advanced metrics
    'wp_before',
    'wp_leverage'
]

TARGET_COLUMN = 'went_for_it'

print(f"Configuration set:")
print(f"  - Random State: {RANDOM_STATE}")
print(f"  - Test Size: {TEST_SIZE}")
print(f"  - CV Folds: {CV_FOLDS}")
print(f"  - Min Coach Opportunities: {MIN_COACH_OPPORTUNITIES}")
print(f"  - Number of Features: {len(FEATURE_COLUMNS)}")

Configuration set:
  - Random State: 42
  - Test Size: 0.2
  - CV Folds: 5
  - Min Coach Opportunities: 20
  - Number of Features: 31


## 3. Data Loading & Preprocessing

In [21]:
def load_and_preprocess_data(filepath: str) -> tuple:
    """Load data and prepare for modeling."""
    
    print("=" * 60)
    print("LOADING DATA")
    print("=" * 60)
    
    df = pd.read_csv(filepath)
    df.dropna(inplace = True)
    print(f"Loaded {len(df):,} records")
    
    # Basic info
    print(f"\nTarget distribution:")
    print(f"  Went for it: {df[TARGET_COLUMN].sum():,} ({df[TARGET_COLUMN].mean()*100:.2f}%)")
    print(f"  Punt/FG: {(df[TARGET_COLUMN]==0).sum():,} ({(1-df[TARGET_COLUMN].mean())*100:.2f}%)")
    
    # Select features that exist in the data
    available_features = [f for f in FEATURE_COLUMNS if f in df.columns]
    missing_features = [f for f in FEATURE_COLUMNS if f not in df.columns]
    
    if missing_features:
        print(f"\nWarning: Missing features: {missing_features}")
    
    print(f"\nUsing {len(available_features)} features for modeling")
    
    # Prepare feature matrix
    X = df[available_features].copy()
    y = df[TARGET_COLUMN].copy()
    
    # Handle any remaining NaN values
    X = X.dropna()
    X.drop(columns="log_distance", inplace=True)
    
    # Store metadata for later
    metadata = df[['id_play', 'game_id', 'year', 'week', 'pos_team', 
                   'def_pos_team', 'coach_name', 'play_type', 'distance',
                   'yards_to_goal', TARGET_COLUMN]].copy()
    
    return X, y, metadata, available_features

In [22]:
# Load the data
X, y, metadata, feature_names = load_and_preprocess_data('Fourth_Down_Model_Data_v2.csv')

LOADING DATA
Loaded 200,056 records

Target distribution:
  Went for it: 43,151 (21.57%)
  Punt/FG: 156,905 (78.43%)

Using 31 features for modeling


In [23]:
# Preview the features
print("Feature Preview:")
X.head(10)

Feature Preview:


,distance,yards_to_goal,period,seconds_remaining_game,seconds_remaining_half,point_differential,total_score,is_trailing,is_leading,abs_point_diff,...,scaled_talent_gap,late_and_close,garbage_time,two_minute_drill,is_conference_game,pos_team_is_home,prev_play_successful,prev_4th_down_success_rate,wp_before,wp_leverage
0,18,76,1,2471,671,0,0,0,0,0,...,1.242744,0,0,0,0,1,0,0.5,0.390406,0.109594
1,6,45,1,2275,475,0,0,0,0,0,...,-1.242744,0,0,0,0,0,0,0.5,0.483306,0.016694
2,1,37,1,2154,354,7,7,0,1,7,...,1.242744,0,0,0,0,1,1,0.5,0.547828,0.047828
3,2,60,1,2011,211,-7,7,1,0,7,...,-1.242744,0,0,0,0,0,1,0.5,0.211788,0.288212
4,7,68,1,1830,30,7,7,0,1,7,...,1.242744,0,0,0,0,1,1,1.0,0.699476,0.199476
5,3,49,2,75,75,-7,21,1,0,7,...,-1.242744,0,0,1,0,0,0,0.0,0.229629,0.270371
6,4,81,2,9,9,7,21,0,1,7,...,1.242744,0,0,1,0,1,0,0.5,0.802259,0.302259
7,18,76,3,1619,719,-7,21,1,0,7,...,-1.242744,0,0,0,0,0,0,0.0,0.144379,0.355621
8,6,71,3,1547,647,-14,28,1,0,14,...,-1.242744,0,0,0,0,0,1,0.0,0.048314,0.451686
9,16,27,3,1371,471,14,28,0,1,14,...,1.242744,0,0,0,0,1,0,0.5,0.935216,0.435216


In [24]:
# Check for any data issues
print("Feature Statistics:")
X.describe().T

Feature Statistics:


,count,mean,std,min,25%,50%,75%,max
distance,200056.0,7.691121,5.727653,-3.000000,3.000000,7.000000,10.000000,84.000000
yards_to_goal,200056.0,50.550731,24.147301,-17.000000,32.000000,54.000000,70.000000,100.000000
period,200056.0,2.521984,1.118835,1.000000,2.000000,2.000000,4.000000,9.000000
seconds_remaining_game,200056.0,1070.117652,797.900946,0.000000,393.000000,875.000000,1723.000000,3480.000000
seconds_remaining_half,200056.0,447.343529,272.234881,0.000000,207.000000,447.000000,681.000000,2940.000000
point_differential,200056.0,-2.016840,15.647403,-104.000000,-10.000000,0.000000,7.000000,100.000000
total_score,200056.0,26.991547,19.981969,-57.000000,10.000000,24.000000,41.000000,180.000000
is_trailing,200056.0,0.476217,0.499435,0.000000,0.000000,0.000000,1.000000,1.000000
is_leading,200056.0,0.372111,0.483369,0.000000,0.000000,0.000000,1.000000,1.000000
abs_point_diff,200056.0,11.190472,11.121224,0.000000,3.000000,7.000000,16.000000,104.000000


## 4. Train/Test Split

In [25]:
def split_data(X: pd.DataFrame, y: pd.Series) -> tuple:
    """Split data into train and test sets."""
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=TEST_SIZE, 
        random_state=RANDOM_STATE,
        stratify=y
    )
    
    print(f"Data split:")
    print(f"  Training: {len(X_train):,} samples ({len(X_train)/len(X)*100:.1f}%)")
    print(f"  Testing: {len(X_test):,} samples ({len(X_test)/len(X)*100:.1f}%)")
    print(f"\nTarget balance in training: {y_train.mean()*100:.2f}% went for it")
    print(f"Target balance in testing: {y_test.mean()*100:.2f}% went for it")
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data(X, y)

Data split:
  Training: 160,044 samples (80.0%)
  Testing: 40,012 samples (20.0%)

Target balance in training: 21.57% went for it
Target balance in testing: 21.57% went for it


## 5. Model Definitions

In [26]:
def get_models() -> dict:
    """Define all models to test."""
    
    models = {
        'Logistic Regression': LogisticRegression(
            random_state=RANDOM_STATE, max_iter=1000
        ),
        'Random Forest': RandomForestClassifier(
            n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1
        ),
        'Gradient Boosting': GradientBoostingClassifier(
            n_estimators=100, random_state=RANDOM_STATE
        ),
        'Extra Trees': ExtraTreesClassifier(
            n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1
        ),
        'AdaBoost': AdaBoostClassifier(
            n_estimators=100, random_state=RANDOM_STATE
        ),
        'Decision Tree': DecisionTreeClassifier(
            random_state=RANDOM_STATE
        ),
        'K-Nearest Neighbors': KNeighborsClassifier(
            n_neighbors=5, n_jobs=-1
        ),
        'Naive Bayes': GaussianNB(),
    }
    
    # Add XGBoost if available
    if XGBOOST_AVAILABLE:
        models['XGBoost'] = XGBClassifier(
            n_estimators=100,
            random_state=RANDOM_STATE,
            use_label_encoder=False,
            eval_metric='logloss',
            n_jobs=-1
        )
    
    # Add LightGBM if available
    if LIGHTGBM_AVAILABLE:
        models['LightGBM'] = LGBMClassifier(
            n_estimators=100,
            random_state=RANDOM_STATE,
            verbose=-1,
            n_jobs=-1
        )
    
    print(f"Models to evaluate: {len(models)}")
    for name in models.keys():
        print(f"  • {name}")
    
    return models

models = get_models()

Models to evaluate: 10
  • Logistic Regression
  • Random Forest
  • Gradient Boosting
  • Extra Trees
  • AdaBoost
  • Decision Tree
  • K-Nearest Neighbors
  • Naive Bayes
  • XGBoost
  • LightGBM


## 6. Model Evaluation

In [27]:
def evaluate_models(models: dict, X_train: pd.DataFrame, X_test: pd.DataFrame,
                   y_train: pd.Series, y_test: pd.Series) -> pd.DataFrame:
    """Train and evaluate all models."""
    
    print("\n" + "=" * 60)
    print("MODEL EVALUATION")
    print("=" * 60)
    
    # Scale features for models that benefit from it
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    results = []
    trained_models = {}
    
    for name, model in models.items():
        print(f"\n▶ Training {name}...")
        
        try:
            # Use scaled data for certain models
            if name in ['Logistic Regression', 'K-Nearest Neighbors', 'SVM']:
                model.fit(X_train_scaled, y_train)
                y_pred = model.predict(X_test_scaled)
                y_prob = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
            else:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            roc_auc = roc_auc_score(y_test, y_prob) if y_prob is not None else None
            
            # Cross-validation score
            cv_scores = cross_val_score(model, X_train, y_train, cv=CV_FOLDS, scoring='f1')
            cv_mean = cv_scores.mean()
            cv_std = cv_scores.std()
            
            results.append({
                'Model': name,
                'Accuracy': accuracy,
                'F1-Score': f1,
                'Precision': precision,
                'Recall': recall,
                'ROC-AUC': roc_auc,
                'CV F1 Mean': cv_mean,
                'CV F1 Std': cv_std
            })
            
            trained_models[name] = model
            
            print(f"  ✓ Accuracy: {accuracy:.4f} | F1: {f1:.4f} | CV F1: {cv_mean:.4f} (±{cv_std:.4f})")
            
        except Exception as e:
            print(f"  ✗ Error training {name}: {e}")
    
    # Create results DataFrame
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('F1-Score', ascending=False)
    
    return results_df, trained_models, scaler

In [28]:
# Train and evaluate all models
results_df, trained_models, scaler = evaluate_models(
    models, X_train, X_test, y_train, y_test
)


MODEL EVALUATION

▶ Training Logistic Regression...
  ✓ Accuracy: 0.8464 | F1: 0.5895 | CV F1: 0.5776 (±0.0076)

▶ Training Random Forest...
  ✓ Accuracy: 0.8964 | F1: 0.7459 | CV F1: 0.7347 (±0.0039)

▶ Training Gradient Boosting...
  ✓ Accuracy: 0.8770 | F1: 0.6829 | CV F1: 0.6774 (±0.0052)

▶ Training Extra Trees...
  ✓ Accuracy: 0.8920 | F1: 0.7324 | CV F1: 0.7194 (±0.0046)

▶ Training AdaBoost...
  ✓ Accuracy: 0.8539 | F1: 0.6079 | CV F1: 0.6076 (±0.0034)

▶ Training Decision Tree...
  ✓ Accuracy: 0.8495 | F1: 0.6510 | CV F1: 0.6380 (±0.0028)

▶ Training K-Nearest Neighbors...
  ✓ Accuracy: 0.8498 | F1: 0.6258 | CV F1: 0.4114 (±0.0025)

▶ Training Naive Bayes...
  ✓ Accuracy: 0.7819 | F1: 0.5495 | CV F1: 0.5449 (±0.0049)

▶ Training XGBoost...
  ✓ Accuracy: 0.8903 | F1: 0.7331 | CV F1: 0.7291 (±0.0066)

▶ Training LightGBM...
  ✓ Accuracy: 0.8873 | F1: 0.7261 | CV F1: 0.7238 (±0.0068)


In [29]:
# Display model comparison
print("\n" + "=" * 60)
print("MODEL COMPARISON RESULTS")
print("=" * 60 + "\n")

# Format for display
display_df = results_df.copy()
for col in ['Accuracy', 'F1-Score', 'Precision', 'Recall', 'ROC-AUC', 'CV F1 Mean', 'CV F1 Std']:
    if col in display_df.columns:
        display_df[col] = display_df[col].apply(lambda x: f"{x:.4f}" if pd.notnull(x) else "N/A")

display_df


MODEL COMPARISON RESULTS



,Model,Accuracy,F1-Score,Precision,Recall,ROC-AUC,CV F1 Mean,CV F1 Std
1,Random Forest,0.8964,0.7459,0.7915,0.7052,0.9356,0.7347,0.0039
8,XGBoost,0.8903,0.7331,0.7712,0.6986,0.9319,0.7291,0.0066
3,Extra Trees,0.8920,0.7324,0.7869,0.6849,0.9330,0.7194,0.0046
9,LightGBM,0.8873,0.7261,0.7633,0.6924,0.9301,0.7238,0.0068
2,Gradient Boosting,0.8770,0.6829,0.7691,0.6140,0.9190,0.6774,0.0052
5,Decision Tree,0.8495,0.6510,0.6511,0.6509,0.7778,0.6380,0.0028
6,K-Nearest Neighbors,0.8498,0.6258,0.6762,0.5824,0.8562,0.4114,0.0025
4,AdaBoost,0.8539,0.6079,0.7220,0.5250,0.8909,0.6076,0.0034
0,Logistic Regression,0.8464,0.5895,0.6961,0.5112,0.8748,0.5776,0.0076
7,Naive Bayes,0.7819,0.5495,0.4955,0.6168,0.8187,0.5449,0.0049


In [31]:
# Identify best model
best_model_name = results_df.iloc[0]['Model']
best_model = trained_models[best_model_name]

print("\n" + "=" * 60)
print(f"BEST MODEL: {best_model_name}")
print("=" * 60)
print(f"  Accuracy:  {results_df.iloc[0]['Accuracy']:.4f}")
print(f"  F1-Score:  {results_df.iloc[0]['F1-Score']:.4f}")
print(f"  Precision: {results_df.iloc[0]['Precision']:.4f}")
print(f"  Recall:    {results_df.iloc[0]['Recall']:.4f}")
print(f"  ROC-AUC:   {results_df.iloc[0]['ROC-AUC']:.4f}")


BEST MODEL: Random Forest
  Accuracy:  0.8964
  F1-Score:  0.7459
  Precision: 0.7915
  Recall:    0.7052
  ROC-AUC:   0.9356


In [ ]:
# Save initial results
results_df.to_csv('model_comparison_results.csv', index=False)
print("✓ Saved: model_comparison_results.csv")

## 7. Hyperparameter Tuning

In [32]:
def get_param_grid(model_name: str) -> dict:
    """Get hyperparameter grid for the specified model."""
    
    param_grids = {
        'XGBoost': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7, 9],
            'learning_rate': [0.01, 0.05, 0.1],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0],
            'min_child_weight': [1, 3, 5]
        },
        'LightGBM': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7, -1],
            'learning_rate': [0.01, 0.05, 0.1],
            'num_leaves': [31, 50, 100],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0]
        },
        'Random Forest': {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, 30, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2', None]
        },
        'Gradient Boosting': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.05, 0.1],
            'subsample': [0.8, 0.9, 1.0],
            'min_samples_split': [2, 5, 10]
        },
        'Logistic Regression': {
            'C': [0.001, 0.01, 0.1, 1, 10],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear', 'saga']
        }
    }
    
    return param_grids.get(model_name, {})

In [33]:
def tune_model(model_name: str, model, X_train: pd.DataFrame, y_train: pd.Series,
               scaler: StandardScaler = None) -> tuple:
    """Perform hyperparameter tuning on the best model."""
    
    print("\n" + "=" * 60)
    print(f"HYPERPARAMETER TUNING: {model_name}")
    print("=" * 60)
    
    param_grid = get_param_grid(model_name)
    
    if not param_grid:
        print(f"No hyperparameter grid defined for {model_name}")
        print("Returning original model...")
        return model, {}
    
    # Prepare data
    if model_name in ['Logistic Regression'] and scaler is not None:
        X_train_tuning = scaler.fit_transform(X_train)
    else:
        X_train_tuning = X_train
    
    # Create new model instance for tuning
    if model_name == 'XGBoost' and XGBOOST_AVAILABLE:
        base_model = XGBClassifier(
            random_state=RANDOM_STATE,
            use_label_encoder=False,
            eval_metric='logloss',
            n_jobs=-1
        )
    elif model_name == 'LightGBM' and LIGHTGBM_AVAILABLE:
        base_model = LGBMClassifier(
            random_state=RANDOM_STATE,
            verbose=-1,
            n_jobs=-1
        )
    elif model_name == 'Random Forest':
        base_model = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)
    elif model_name == 'Gradient Boosting':
        base_model = GradientBoostingClassifier(random_state=RANDOM_STATE)
    elif model_name == 'Logistic Regression':
        base_model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
    else:
        print(f"Using pre-trained model for {model_name}")
        return model, {}
    
    # Use smaller grid for faster tuning
    reduced_grid = {k: v[:3] if len(v) > 3 else v for k, v in param_grid.items()}
    
    print(f"\nSearching {len(reduced_grid)} parameters...")
    print(f"Grid: {reduced_grid}")
    
    grid_search = GridSearchCV(
        base_model,
        reduced_grid,
        cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE),
        scoring='f1',
        n_jobs=-1,
        verbose=1
    )
    
    grid_search.fit(X_train_tuning, y_train)
    
    print(f"\n✓ Best parameters: {grid_search.best_params_}")
    print(f"✓ Best CV F1-Score: {grid_search.best_score_:.4f}")
    
    return grid_search.best_estimator_, grid_search.best_params_

In [34]:
# Tune the best model
tuned_model, best_params = tune_model(
    best_model_name, best_model, X_train, y_train, scaler
)


HYPERPARAMETER TUNING: Random Forest

Searching 5 parameters...
Grid: {'n_estimators': [100, 200, 300], 'max_depth': [10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'max_features': ['sqrt', 'log2', None]}
Fitting 3 folds for each of 243 candidates, totalling 729 fits

✓ Best parameters: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
✓ Best CV F1-Score: 0.7319


## 8. Final Model Evaluation

In [35]:
def evaluate_final_model(model, X_test: pd.DataFrame, y_test: pd.Series,
                         feature_names: list, scaler: StandardScaler = None,
                         model_name: str = "Model") -> dict:
    """Comprehensive evaluation of the final tuned model."""
    
    print("\n" + "=" * 60)
    print(f"FINAL MODEL EVALUATION: {model_name}")
    print("=" * 60)
    
    # Prepare test data
    if model_name in ['Logistic Regression'] and scaler is not None:
        X_test_eval = scaler.transform(X_test)
    else:
        X_test_eval = X_test
    
    # Predictions
    y_pred = model.predict(X_test_eval)
    y_prob = model.predict_proba(X_test_eval)[:, 1]
    
    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)
    
    print(f"\nFinal Metrics:")
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  ROC-AUC:   {roc_auc:.4f}")
    
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['Punt/FG', 'Go For It']))
    
    print(f"Confusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    print(f"  [[TN={cm[0,0]:,}  FP={cm[0,1]:,}]")
    print(f"   [FN={cm[1,0]:,}  TP={cm[1,1]:,}]]")
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'roc_auc': roc_auc
    }

In [36]:
# Evaluate the final tuned model
final_metrics = evaluate_final_model(
    tuned_model, X_test, y_test, feature_names, scaler, best_model_name
)


FINAL MODEL EVALUATION: Random Forest

Final Metrics:
  Accuracy:  0.8967
  F1-Score:  0.7470
  Precision: 0.7916
  Recall:    0.7071
  ROC-AUC:   0.9372

Classification Report:
              precision    recall  f1-score   support

     Punt/FG       0.92      0.95      0.94     31382
   Go For It       0.79      0.71      0.75      8630

    accuracy                           0.90     40012
   macro avg       0.86      0.83      0.84     40012
weighted avg       0.89      0.90      0.89     40012

Confusion Matrix:
  [[TN=29,776  FP=1,606]
   [FN=2,528  TP=6,102]]


In [37]:
# Feature importance (if available)
if hasattr(tuned_model, 'feature_importances_'):
    print("\n" + "=" * 60)
    print("FEATURE IMPORTANCES")
    print("=" * 60)
    
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': tuned_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nTop 15 Most Important Features:")
    for i, row in importance_df.head(15).iterrows():
        bar = "█" * int(row['Importance'] * 50)
        print(f"  {row['Feature']:<30} {row['Importance']:.4f} {bar}")
    
    importance_df


FEATURE IMPORTANCES


ValueError: All arrays must be of the same length

## 9. Calculate Aggression Over Expectation (AOE)

In [38]:
def calculate_aggression_scores(model, X: pd.DataFrame, metadata: pd.DataFrame,
                                feature_names: list, scaler: StandardScaler = None,
                                model_name: str = "Model") -> pd.DataFrame:
    """Calculate Aggression Over Expectation for each coach."""
    
    print("\n" + "=" * 60)
    print("CALCULATING AGGRESSION OVER EXPECTATION")
    print("=" * 60)
    
    # Prepare full data for prediction
    if model_name in ['Logistic Regression'] and scaler is not None:
        X_pred = scaler.transform(X)
    else:
        X_pred = X
    
    # Get predictions for all plays
    predictions = model.predict(X_pred)
    probabilities = model.predict_proba(X_pred)[:, 1]
    
    # Add predictions to metadata
    results = metadata.copy()
    results['predicted_go'] = predictions
    results['predicted_prob'] = probabilities
    results['actual_go'] = results['went_for_it']
    
    # Calculate play-level difference
    results['go_over_expected'] = results['actual_go'] - results['predicted_go']
    results['go_over_expected_prob'] = results['actual_go'] - results['predicted_prob']
    
    print(f"\nPredictions generated for {len(results):,} plays")
    
    # Aggregate by coach
    coach_aggression = results.groupby('coach_name').agg({
        'actual_go': ['sum', 'mean', 'count'],
        'predicted_go': ['sum', 'mean'],
        'predicted_prob': ['sum', 'mean'],
        'go_over_expected': ['sum', 'mean'],
        'go_over_expected_prob': ['sum', 'mean'],
        'pos_team': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Unknown',
        'year': ['min', 'max']
    }).reset_index()
    
    # Flatten column names
    coach_aggression.columns = [
        'coach_name',
        'total_go_for_it', 'actual_go_rate', 'total_opportunities',
        'expected_go_for_it', 'expected_go_rate',
        'expected_go_prob_sum', 'expected_go_prob_rate',
        'go_over_expected_sum', 'go_over_expected_rate',
        'go_over_expected_prob_sum', 'go_over_expected_prob_rate',
        'primary_team', 'first_year', 'last_year'
    ]
    
    # Calculate Aggression Over Expectation (AOE)
    coach_aggression['aggression_over_expected'] = (
        coach_aggression['actual_go_rate'] - coach_aggression['expected_go_rate']
    )
    
    # Probability-based version
    coach_aggression['aggression_over_expected_prob'] = (
        coach_aggression['actual_go_rate'] - coach_aggression['expected_go_prob_rate']
    )
    
    # Filter to coaches with minimum opportunities
    coach_aggression_filtered = coach_aggression[
        coach_aggression['total_opportunities'] >= MIN_COACH_OPPORTUNITIES
    ].copy()
    
    # Sort by aggression
    coach_aggression_filtered = coach_aggression_filtered.sort_values(
        'aggression_over_expected', ascending=False
    )
    
    # Add rank
    coach_aggression_filtered['aggression_rank'] = range(1, len(coach_aggression_filtered) + 1)
    
    print(f"\nCoaches with {MIN_COACH_OPPORTUNITIES}+ opportunities: {len(coach_aggression_filtered)}")
    
    return coach_aggression_filtered, results

In [39]:
# Calculate aggression scores
coach_aggression, play_results = calculate_aggression_scores(
    tuned_model, X, metadata, feature_names, scaler, best_model_name
)


CALCULATING AGGRESSION OVER EXPECTATION

Predictions generated for 200,056 plays

Coaches with 20+ opportunities: 367


In [40]:
# Display Top 10 Most Aggressive Coaches
print("\n" + "=" * 80)
print("TOP 10 MOST AGGRESSIVE COACHES")
print("=" * 80)

top10 = coach_aggression.head(10)[[
    'aggression_rank', 'coach_name', 'primary_team', 'total_opportunities',
    'actual_go_rate', 'expected_go_rate', 'aggression_over_expected'
]]

for _, row in top10.iterrows():
    print(f"{row['aggression_rank']:2}. {row['coach_name']:<25} ({row['primary_team']:<20}) "
          f"Opps: {row['total_opportunities']:4} | "
          f"Actual: {row['actual_go_rate']:.3f} | "
          f"Expected: {row['expected_go_rate']:.3f} | "
          f"AOE: {row['aggression_over_expected']:+.3f}")


TOP 10 MOST AGGRESSIVE COACHES
 1. Willie Simmons            (Florida International) Opps:  118 | Actual: 0.254 | Expected: 0.203 | AOE: +0.051
 2. Joe Harasymiak            (Massachusetts       ) Opps:  125 | Actual: 0.224 | Expected: 0.176 | AOE: +0.048
 3. Jason Eck                 (New Mexico          ) Opps:   95 | Actual: 0.337 | Expected: 0.295 | AOE: +0.042
 4. Joey McGuire              (Texas Tech          ) Opps:  412 | Actual: 0.303 | Expected: 0.265 | AOE: +0.039
 5. Lane Kiffin               (Ole Miss            ) Opps:  850 | Actual: 0.321 | Expected: 0.285 | AOE: +0.036
 6. Dan Lanning               (Oregon              ) Opps:  301 | Actual: 0.306 | Expected: 0.272 | AOE: +0.033
 7. Mike London               (Virginia            ) Opps:   95 | Actual: 0.158 | Expected: 0.126 | AOE: +0.032
 8. Trent Dilfer              (UAB                 ) Opps:  288 | Actual: 0.323 | Expected: 0.292 | AOE: +0.031
 9. Dave Aranda               (Baylor              ) Opps:  593 | Actua

In [41]:
# Display Top 10 Most Conservative Coaches
print("\n" + "=" * 80)
print("TOP 10 MOST CONSERVATIVE COACHES")
print("=" * 80)

bottom10 = coach_aggression.tail(10)[[
    'aggression_rank', 'coach_name', 'primary_team', 'total_opportunities',
    'actual_go_rate', 'expected_go_rate', 'aggression_over_expected'
]].iloc[::-1]

for _, row in bottom10.iterrows():
    print(f"{row['aggression_rank']:2}. {row['coach_name']:<25} ({row['primary_team']:<20}) "
          f"Opps: {row['total_opportunities']:4} | "
          f"Actual: {row['actual_go_rate']:.3f} | "
          f"Expected: {row['expected_go_rate']:.3f} | "
          f"AOE: {row['aggression_over_expected']:+.3f}")


TOP 10 MOST CONSERVATIVE COACHES
367. Scott Shafer              (Syracuse            ) Opps:  107 | Actual: 0.131 | Expected: 0.159 | AOE: -0.028
366. David Bailiff             (Rice                ) Opps:  291 | Actual: 0.162 | Expected: 0.186 | AOE: -0.024
365. Ryan Beard                (Missouri State      ) Opps:   93 | Actual: 0.140 | Expected: 0.161 | AOE: -0.022
364. Urban Meyer               (Ohio State          ) Opps:  362 | Actual: 0.213 | Expected: 0.232 | AOE: -0.019
363. Tony Gibson               (Marshall            ) Opps:  108 | Actual: 0.139 | Expected: 0.157 | AOE: -0.019
362. Lovie Smith               (Illinois            ) Opps:  483 | Actual: 0.133 | Expected: 0.149 | AOE: -0.017
361. Gary Patterson            (TCU                 ) Opps:  696 | Actual: 0.135 | Expected: 0.148 | AOE: -0.013
360. Nick Saban                (Alabama             ) Opps:  712 | Actual: 0.170 | Expected: 0.183 | AOE: -0.013
359. Mark Richt                (Miami               ) Opps:  3

In [42]:
# Full rankings table
print("\n" + "=" * 60)
print("FULL AGGRESSION RANKINGS")
print("=" * 60)

coach_aggression[[
    'aggression_rank', 'coach_name', 'primary_team', 
    'total_opportunities', 'actual_go_rate', 'expected_go_rate', 
    'aggression_over_expected'
]].head(25)


FULL AGGRESSION RANKINGS


,aggression_rank,coach_name,primary_team,total_opportunities,actual_go_rate,expected_go_rate,aggression_over_expected
363,1,Willie Simmons,Florida International,118,0.254237,0.203390,0.050847
166,2,Joe Harasymiak,Massachusetts,125,0.224000,0.176000,0.048000
139,3,Jason Eck,New Mexico,95,0.336842,0.294737,0.042105
170,4,Joey McGuire,Texas Tech,412,0.303398,0.264563,0.038835
201,5,Lane Kiffin,Ole Miss,850,0.321176,0.284706,0.036471
75,6,Dan Lanning,Oregon,301,0.305648,0.272425,0.033223
245,7,Mike London,Virginia,95,0.157895,0.126316,0.031579
347,8,Trent Dilfer,UAB,288,0.322917,0.291667,0.031250
83,9,Dave Aranda,Baylor,593,0.372681,0.342327,0.030354
53,10,Charles Kelly,Jacksonville State,102,0.235294,0.205882,0.029412


## 10. Save Outputs

In [43]:
print("\n" + "=" * 60)
print("SAVING OUTPUTS")
print("=" * 60)

# Save coach aggression rankings
coach_aggression.to_csv('coach_aggression_rankings.csv', index=False)
print("✓ Saved: coach_aggression_rankings.csv")

# Save play-level predictions
play_results.to_csv('play_level_predictions.csv', index=False)
print("✓ Saved: play_level_predictions.csv")

# Save the final model
joblib.dump(tuned_model, 'final_fourth_down_model.joblib')
print("✓ Saved: final_fourth_down_model.joblib")

# Save scaler if used
if scaler is not None:
    joblib.dump(scaler, 'feature_scaler.joblib')
    print("✓ Saved: feature_scaler.joblib")

# Save final metrics
metrics_df = pd.DataFrame([{
    'model': best_model_name,
    'best_params': str(best_params),
    **final_metrics
}])
metrics_df.to_csv('final_model_metrics.csv', index=False)
print("✓ Saved: final_model_metrics.csv")


SAVING OUTPUTS
✓ Saved: coach_aggression_rankings.csv
✓ Saved: play_level_predictions.csv
✓ Saved: final_fourth_down_model.joblib
✓ Saved: feature_scaler.joblib
✓ Saved: final_model_metrics.csv


## 11. Summary

In [45]:
print("\n" + "=" * 60)
print("PIPELINE COMPLETE")
print("=" * 60)

print(f"\nDataset Summary:")
print(f"  Total plays analyzed: {len(X):,}")
print(f"  Coaches ranked: {len(coach_aggression)}")
print(f"  Years covered: {metadata['year'].min()} - {metadata['year'].max()}")

print(f"\nModel Summary:")
print(f"  Best model: {best_model_name}")
print(f"  F1-Score: {final_metrics['f1']:.4f}")
print(f"  Accuracy: {final_metrics['accuracy']:.4f}")

print(f"\nOutput Files:")
print(f"  • coach_aggression_rankings.csv")
print(f"  • play_level_predictions.csv")
print(f"  • model_comparison_results.csv")
print(f"  • final_fourth_down_model.joblib")
print(f"  • final_model_metrics.csv")

print(f"\nMost Aggressive: {coach_aggression.iloc[0]['coach_name']} (AOE: {coach_aggression.iloc[0]['aggression_over_expected']:+.3f})")
print(f"Most Conservative: {coach_aggression.iloc[-1]['coach_name']} (AOE: {coach_aggression.iloc[-1]['aggression_over_expected']:+.3f})")


PIPELINE COMPLETE

Dataset Summary:
  Total plays analyzed: 200,056
  Coaches ranked: 367
  Years covered: 2015 - 2025

Model Summary:
  Best model: Random Forest
  F1-Score: 0.7470
  Accuracy: 0.8967

Output Files:
  • coach_aggression_rankings.csv
  • play_level_predictions.csv
  • model_comparison_results.csv
  • final_fourth_down_model.joblib
  • final_model_metrics.csv

Most Aggressive: Willie Simmons (AOE: +0.051)
Most Conservative: Scott Shafer (AOE: -0.028)
